In [ ]:
import os
import urllib.request


__all__ = ["download"]


def download_files(bucket_path, file_list, download_path):
    """
    Provide path to s3 bucket, download a file list to download path
    """
    if not os.path.isdir(download_path):
        os.makedirs(download_path)
    for f in file_list:
        # check if file already exists
        file_path = os.path.join(download_path, f)
        if os.path.isfile(file_path):
            print ('File "%s" already exists' %f)
        else:
            print ('Downloading "%s" ...' % f)
            urllib.request.urlretrieve(bucket_path + f, file_path)
            print ('Done')


def download(file_list=[]):
    """
    Downloads files from AWS S3 repository
    Here are all avialble dataset from the repository
    file_list=["yelp_academic_dataset_business.pickle"
               "yelp_academic_dataset_review.pickle",
               "yelp_academic_dataset_user.pickle",
               "yelp_academic_dataset_checkin.pickle",
               "yelp_academic_dataset_tip.pickle"]
    """
    if file_list == []:
        print ("Providiing empty file_list, no download...")
    else:
        bucket_path = "https://s3-us-west-2.amazonaws.com/science-of-science-bucket/yelp_academic_dataset/"
        current_path = os.path.dirname(os.path.abspath("/"))
        download_path = "/Users/UID/BIGDATA/Yelp Analysis"
        download_files(bucket_path, file_list, download_path)


In [ ]:
download(file_list=["yelp_academic_dataset_business.pickle",
                              "yelp_academic_dataset_review.pickle",
                              "yelp_academic_dataset_user.pickle",
                              "yelp_academic_dataset_checkin.pickle",
                              "yelp_academic_dataset_tip.pickle"])

In [18]:
import pandas as pd
from elasticsearch import Elasticsearch 
es = Elasticsearch()

review = pd.read_pickle('C:/Users/UID/BIGDATA/Yelp Analysis/yelp_academic_dataset_business.pickle')
out = review.to_json(orient='records')[1:-1].replace('},{', '} {')

In [19]:

result = es.indices.create(index='yelp', ignore=400)
print(result)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'yelp'}


In [ ]:
b_data = []
for i in review.index:
    d = {}
    for j in review.columns:
        d[j] = review[j][i]
    b_data.append(d)
len(b_data)

In [14]:
review.columns

Index(['attributes', 'business_id', 'categories', 'city', 'full_address',
       'hours', 'latitude', 'longitude', 'name', 'neighborhoods', 'open',
       'review_count', 'stars', 'state', 'type'],
      dtype='object')

In [11]:
from elasticsearch import Elasticsearch 
import json

table = ['attributes', 'business_id', 'categories', 'city', 'full_address',
       'hours', 'latitude', 'longitude', 'name', 'neighborhoods', 'open',
       'review_count', 'stars', 'state', 'type']

i = 0
for table in b_data:
    result=es.create(index='yelp', doc_type='business', id=i, body=b_data)
    i += 1
    if i % 61184 == 0:
        print(i, 'Records have been uploaded')
print(table, 'b_data finished.')


ModuleNotFoundError: No module named 'elasticsearch'